# HW5 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Mar 23 2018

In [ ]:
# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
set_matplotlib_formats('png', 'pdf')
# set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt
from IPython.display import HTML
from log_progress import log_progress

## 1. 

> The world model is shown in figure 1. The robot is a differential drive
system of size 50x50.

1. Generate the configuration space for the robot with a grid size of 2x2 and 5 deg in angular resolution. Generate an illustration of what the configuration space looks like with the robot at orientations 0, 45 and 90 deg.
2. Use greedy search find the shortest path between start-point (50,50) and end-point (750,250). Illustrate the path and provide its length.
3. Compute the safest past from start to finish (hint: medial axis transform). Illustrate the path and provide its length.
4. Use probabilistic roadmaps (PRM) to compute a path between startand end-points with 50, 100 and 500 sample points. What is the difference in path length? Illustrate each computed path.
5. Do the same with Rapid exploring random trees (RRT). What are the main differences in performance between PRM and RRT? Illustrate each path.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import pathlib
import scipy.spatial
import scipy.signal

from matplotlib import animation
from collections import OrderedDict
from PIL import Image

In [ ]:
import warnings
warnings.filterwarnings('ignore')

We'll start by defining a grid-based environment to encapsulate a discretized representation of our world model. We then will populate the 2D grid environment with the set number of wall objects that constrain possible paths what limits our work space.

In [ ]:
scale = 1

width = int(800/scale)
hight = int(300/scale)
seg1 = int(200/scale)
seg2 = int(600/scale)
seg3 = int(400/scale)
seg4 = int(100/scale)

map_2d = np.zeros(shape=(width, hight))
map_2d[0,:] = 1
map_2d[:,0] = 1
map_2d[:,hight-1] = 1
map_2d[width-1,:] = 1

map_2d[seg1-1,0:seg1] = 1
map_2d[seg2-1,0:seg1] = 1

map_2d[seg3-1,hight-1-seg1:hight-1] = 1
map_2d[seg3-1-seg4:seg3-1+seg4,hight-1-seg1] = 1

Well also define our Robot as pixelated shape, subject to the same scale used in deriving the environment. 

In [ ]:
rwidth = int(50/scale)
rhight = int(50/scale)
robot_2d = np.ones(shape=(rwidth, rhight))

Here we’ll define a function to rotate our robot representation given an angle in degrees.

In [ ]:
def rotate_bound(image, angle):
    # borrowed from https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

Given both are world and robot representations I represented as occupancy and 2D arrays, we can use simple 2d convolution, optimized with the FFT  transform,  to generate our configuration space with respect to a given angle, or list of  angles In which the robot is rotated.

In [ ]:
def get_config_2d(map_2d, robot_2d_rot):
    config_2d = scipy.signal.fftconvolve(map_2d, robot_2d_rot, mode='same')
    return config_2d

def get_config_space(map_2d, robot_2d_rot, angles):
    config_space = np.empty((width, hight, angle_res))
    for i, angle in enumerate(angles):
        robot_2d_rot = rotate_bound(robot_2d, angle)
        config_space[:,:,i] = get_config_2d(map_2d, robot_2d_rot)
    return config_space

We also define a simple function to plot our workspace or partial configuration space for visualization.

In [ ]:
def plot_map_2d(map_2d):
    f, axs = plt.subplots(1,1,figsize=(24,64))
#     axs.imshow(map_2d.T, cmap=plt.cm.Spectral_r, interpolation='none')
    axs.imshow(map_2d.T, cmap=plt.cm.nipy_spectral, interpolation='none')
    for d in ["left", "top", "bottom", "right"]:
        axs.spines[d].set_visible(False)
    plt.show()

Here will generate our configuration space for the angles between 0 and 90 degrees at a degree resolution of one. Because we are using scipy’s fftconvolve function that padds our signal to next nearest power of the hamming numbers for the fft function optimized for those radixs. This makes this step quite faster than compared to traditional 2D convolution of signal/kernel or workspace/robot respectively.

In [ ]:
angle_res = 90
angles = np.linspace(start=0, stop=90, num=angle_res, endpoint=False)
config_space = get_config_space(map_2d, robot_2d, angles)

Plotting this grid based world, we can see the layout as shown in the assignment figure. 

In [ ]:
plot_map_2d(map_2d)

In the next three figures, we can visualize the configuration space 4 robot rotations at 0, 45, and 90 degrees respectively. Because our configuration space was acquired using to the convolutions, we can visualize the raw response to provide additional insight in robot collisional intensity, as well as a regionalized cost map.

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 0)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 45)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 90)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

Because our robot is rotationally symmetrical across every half pi radians, we can flatten the configuration space across rotations from 0 to 90 degrees by taking the max value along the 3rd axis on our configuration space 3D array representation. This renders the most conservative configuration space where the black region denote all points in the world reference frame that our robot can freely rotate in place.

This is useful for robotic navigation stacks that commonly make use of rotational recovery behaviors when losing localization or determining alternate routes. The astute reader will notice that this is the same result as in taking the  the 2D deconvolution of the world map  with respect to a kernel that is a  simple Circle with a radius defined as the maximum distance a collision point along the robot geometry is with respect to the center of its rotation (assuming our differential drive model).

In [ ]:
config_space_2d = config_space.max(axis=2)
# config_space_2d = np.clip(a=config_space_2d, a_min=0, a_max=0.001)
plot_map_2d(config_space_2d)

In [ ]:
config_space_2d = config_space.max(axis=2)
config_space_2d = np.clip(a=config_space_2d, a_min=0, a_max=0.001)
plot_map_2d(config_space_2d)

We can define a function to sweep through the configuration space in an animated fashion. However this is quite slow using to matplotlib.

In [ ]:
import matplotlib
# matplotlib.use('GTKAgg')

def plot_3d(config_space, animated=False, stride=1):
#     config_space = config_space[::stride]
    
    fig, axs = plt.subplots(1,1,figsize=(16,6))
    fig.canvas.draw()
    

    def init():
        axs.imshow(config_space[:,:,0].T, cmap=plt.cm.Spectral_r, interpolation='none')
        plt.axis('off')
        plt.tight_layout()
#         for d in ["left", "top", "bottom", "right"]:
#             axs.spines[d].set_visible(False)
        
        return fig,
    
    def animate(i):
        axs.imshow(config_space[:,:,i].T, cmap=plt.cm.Spectral_r, interpolation='none')
        return fig,
    
    if animated:
        frames = log_progress(range(angle_res), 1)
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                       frames=frames, interval=20, blit=True)
        video = anim.to_html5_video()
        return video
    else:
        init()
        return fig

# empty2_video = plot_3d(config_space, animated=True)
# display(HTML(empty2_video))

A better alternative is perhaps using pygtgraph and let OpenGL do the heavy lifting for us in rendering the frames.

In [ ]:
# import numpy as np
# from pyqtgraph.Qt import QtCore, QtGui
# import pyqtgraph as pg

# # Interpret image data as row-major instead of col-major
# pg.setConfigOptions(imageAxisOrder='row-major')

# app = QtGui.QApplication([])

# ## Create window with ImageView widget
# win = QtGui.QMainWindow()
# win.resize(800,300)
# imv = pg.ImageView()
# win.setCentralWidget(imv)
# win.show()
# win.setWindowTitle('pyqtgraph example: ImageView')

# data = config_space[:,:,:]
# # data = np.clip(a=data, a_min=0, a_max=1)


# ## Display the data and assign each frame a time value from 1.0 to 3.0
# imv.setImage(data.T, xvals=angles)

# # Get the colormap
# colormap = plt.cm.nipy_spectral  # cm.get_cmap("CMRmap")
# colormap._init()
# lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
# imv.imageItem.setLookupTable(lut)

# ptr = 0
# def update():
#     global data, ptr
#     imv.imageItem.updateImage(data[:,:,ptr].T)
#     imv.imageItem.setLookupTable(lut)
#     ptr += 1
#     ptr = ptr % angles.size
# timer = QtCore.QTimer()
# timer.timeout.connect(update)

# ## Start Qt event loop unless running in interactive mode.
# if __name__ == '__main__':
#     import sys
#     if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
#         timer.start(50)
#         QtGui.QApplication.instance().exec_()


We can also use pygtgraph to slice the Configuration space arbitrarily, similarly done with MRI scans.

In [ ]:
# import numpy as np
# from pyqtgraph.Qt import QtCore, QtGui
# import pyqtgraph as pg

# app = QtGui.QApplication([])

# ## Create window with two ImageView widgets
# win = QtGui.QMainWindow()
# win.resize(800,800)
# win.setWindowTitle('pyqtgraph example: DataSlicing')
# cw = QtGui.QWidget()
# win.setCentralWidget(cw)
# l = QtGui.QGridLayout()
# cw.setLayout(l)
# imv1 = pg.ImageView()
# imv2 = pg.ImageView()
# l.addWidget(imv1, 0, 0)
# l.addWidget(imv2, 1, 0)
# win.show()

# roi = pg.LineSegmentROI([[10, 64], [120,64]], pen='r')
# imv1.addItem(roi)

# x1 = np.linspace(-30, 10, 128)[:, np.newaxis, np.newaxis]
# x2 = np.linspace(-20, 20, 128)[:, np.newaxis, np.newaxis]
# y = np.linspace(-30, 10, 128)[np.newaxis, :, np.newaxis]
# z = np.linspace(-20, 20, 128)[np.newaxis, np.newaxis, :]
# d1 = np.sqrt(x1**2 + y**2 + z**2)
# d2 = 2*np.sqrt(x1[::-1]**2 + y**2 + z**2)
# d3 = 4*np.sqrt(x2**2 + y[:,::-1]**2 + z**2)
# # data = (np.sin(d1) / d1**2) + (np.sin(d2) / d2**2) + (np.sin(d3) / d3**2)
# data = config_space[:,:,:]

# def update():
#     global data, imv1, imv2
#     d2 = roi.getArrayRegion(data, imv1.imageItem, axes=(1,2))
#     imv2.setImage(d2)
    
# roi.sigRegionChanged.connect(update)


# ## Display the data
# imv1.setImage(data)
# imv1.setHistogramRange(-0.01, 0.01)
# imv1.setLevels(-0.003, 0.003)

# update()

# ## Start Qt event loop unless running in interactive mode.
# if __name__ == '__main__':
#     import sys
#     if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
#         QtGui.QApplication.instance().exec_()

Lastly we can export the occupancy grid from our configuration workspace and to a three dimensional Point cloud. Obviously this only works because of the 2D dimension of our initial world frame with the additional dimension of rotation. However the same slicing technique, as above, could be used to introspect the configuration space for higher dimensions.

In [ ]:
# from plyfile import PlyData, PlyElement
# data = np.clip(a=config_space, a_min=0, a_max=1)
# points = np.asarray(np.where(data == 1)).T
# vertex = np.array(list(map(tuple, points)), dtype=[('x', 'u2'), ('y', 'u2'), ('z', 'u2')])
# el = PlyElement.describe(vertex, 'config_space')
# PlyData([el]).write('config_space.ply')

In [ ]:
# # import numpy as np
# import yaml
# matrix = np.random.randint(2, size=(10,7))
# with open('foo.yaml', 'w') as f:
#     yaml.dump(config_space.flatten().tolist(), f)

In [ ]:
# thresh = 0.001
# config_space_bool = config_space.copy()
# config_space_bool[config_space < thresh] = 0
# config_space_bool[config_space >= thresh] = 1
# plot_map_2d(config_space_bool[:,:,45])

In [ ]:
# np.savetxt('config_space_bool.yaml', config_space_bool.flatten(), delimiter=',', fmt='%d,')

Computing the safest is path through our workspace environment will in this case equate as being the trajectory that is  maximally distant from all nearby obstacles. Essentially this necessitates computing the voronoi partitions for the empty regions in the work space. We can then travers the set of points generated from this tessellation as it provides us a simplistic roadmap. We then need only find the closest decomposed points to both the start and the goal. As a final check, we can sample the resulting trajectory using our configuration space as a collisions look up.

In [ ]:
points = np.asarray(np.where(map_2d == 1)).T
vor = scipy.spatial.Voronoi(points)

vertices = vor.vertices[::1].T.astype(int)
map_2d_vert = map_2d.copy()
map_2d_vert[vertices[0],vertices[1]] = 2

plot_map_2d(map_2d_vert)